# Mean IOU for Semantic Segmetation Model
- [Brinkley97/PixelLib model GitHub Repo](https://github.com/Brinkley97/sem_seg_pxLib_ade20k/blob/main/ssForImg.ipynb)

In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from PIL import Image

## Load Semantic Segmetation Images
- Update method to load multiple images

In [2]:
base = "/Users/brinkley97/Documents/development/catalyst_robotics/iou/"
ground_truth_path = base + "ground_truth_rural_input_images/"
ground_truth_directory = os.listdir("ground_truth_rural_input_images/")

In [35]:
def calc_miou(ground_truth_image, prediction_model_image):
    '''Determine the intersection over union when given a ground truth image and prediction model image
    
    Arguments:
    ground_truth_image -- string
    prediction_model_image -- string
    
    Return:
    iou
    
    '''
    
    # print(ground_truth_image)
    # print(prediction_model_image)
    
    '''
    Pre process ground_truth image for MeanIOU
    Get RGB to pass into TF MeanIOU function
    '''
    open_ground_truth_image = Image.open(ground_truth_image)
    ground_truth_rgb = np.asarray(open_ground_truth_image)
    print("****ground_truth_rgb size:", np.shape(ground_truth_rgb))
#    print("ground_truth_rgb image:", ground_truth_rgb)
    
    '''
    Pre process prediction_model image for MeanIOU
    Get RGB to pass into TF MeanIOU function
    '''
    open_prediction_model_image = Image.open(prediction_model_image)
    prediction_model_rgb = np.asarray(open_prediction_model_image)
    # print("****pre-reshape prediction_model_rgb size:", np.shape(prediction_model_rgb))
#    print("prediction_model_rgb image:", prediction_model_rgb)
    
    if np.shape(ground_truth_rgb) != np.shape(prediction_model_rgb):
        # print(True)
        resized_prediction_model_rgb = np.resize(prediction_model_rgb, np.shape(ground_truth_rgb))
        # print("resized_prediction_model_rgb size:", np.shape(resized_prediction_model_rgb))
    else:
        resized_prediction_model_rgb = prediction_model_rgb
    
    '''
    Compute the mean intersection over union with TF
    https://www.tensorflow.org/api_docs/python/tf/keras/metrics/MeanIoU
    update_state method: https://github.com/keras-team/keras/blob/v2.10.0/keras/metrics/metrics.py#L2686-L2736
    '''
    
    mean_iou = tf.keras.metrics.MeanIoU(num_classes=25)
    print(mean_iou)
    # print("prediction_model_rgb size:", np.shape(prediction_model_rgb), prediction_model_rgb)
    # print("resized_prediction_model_rgb size:", np.shape(resized_prediction_model_rgb), resized_prediction_model_rgb)
    mean_iou.update_state(ground_truth_rgb, resized_prediction_model_rgb)
    
    return mean_iou.result().numpy()

In [36]:
def compare_two_images(stored_images):
    '''Pass stored images into the calc_miou function
    
    Arguments:
    stored_images -- dict
    
    Functions:
    calc_miou
    
    Return:
    iou for each set of images -- list
    '''

    store_iou = []
    for key, value in stored_images.items():
        ground_truth_image = key
        print("The ground_truth_image is :", ground_truth_image)
        
        prediction_model_image = value
        print("The prediction_model_image is : ", prediction_model_image)
        
        i_o_u = calc_miou(ground_truth_image, prediction_model_image)
        print("The Mean IOU for these two images : ", i_o_u, "\n")
        store_iou.append(i_o_u)
        
    return store_iou

In [37]:
def models_to_use(base, ground_truth_path, prediction_model_path, ground_truth_directory, prediction_model_directory):  
    '''Set up path to get images
    
    Arguments:
    base -- string
    ground_truth_path -- string
    prediction_model_path -- string
    ground_truth_directory -- list
    prediction_model_directory -- list
    
    Functions:
    calc_iou
    
    Return:
    iou for each set of images
    
    '''
    
    ground_truth_path += '{0}'
    prediction_model_path += '{0}'
    
    ground_truth_images = [ground_truth_path.format(idx) for idx in ground_truth_directory]
    prediction_model_images = [prediction_model_path.format(idx) for idx in prediction_model_directory]
    # print(ground_truth_images, '\n', prediction_model_images)
    
#    Store ground_truth_images as key and m_seg_images as value
    store_images = {}
    
    
    for idx in range(len(ground_truth_images)):
        store_images.update({ground_truth_images[idx]: prediction_model_images[idx]})
    
    miou_per_set_of_images = compare_two_images(store_images)
    return miou_per_set_of_images

In [38]:
m_seg_path = base + "m_seg_rural_input_images/"
m_seg_directory = os.listdir("m_seg_rural_input_images/")

iou_with_m_seg = models_to_use(base, ground_truth_path, m_seg_path, ground_truth_directory, m_seg_directory)

The ground_truth_image is : /Users/brinkley97/Documents/development/catalyst_robotics/iou/ground_truth_rural_input_images/trail-6_00001.png
The prediction_model_image is :  /Users/brinkley97/Documents/development/catalyst_robotics/iou/m_seg_rural_input_images/trail-6_00001.png
****ground_truth_rgb size: (550, 688, 3)
MeanIoU(num_classes=25,name=mean_io_u_8,dtype=float32)
prediction_model_rgb size: (449, 558, 4) [[[255 255 255   0]
  [255 255 255   0]
  [255 255 255   0]
  ...
  [255 255 255   0]
  [255 255 255   0]
  [255 255 255   0]]

 [[255 255 255   0]
  [255 255 255   0]
  [255 255 255   0]
  ...
  [255 255 255   0]
  [255 255 255   0]
  [255 255 255   0]]

 [[255 255 255   0]
  [255 255 255   0]
  [255 255 255   0]
  ...
  [255 255 255   0]
  [255 255 255   0]
  [255 255 255   0]]

 ...

 [[255 255 255   0]
  [255 255 255   0]
  [255 255 255   0]
  ...
  [255 255 255   0]
  [255 255 255   0]
  [255 255 255   0]]

 [[255 255 255   0]
  [255 255 255   0]
  [255 255 255   0]
  ...
 

InvalidArgumentError: `labels` out of bound
Condition x < y did not hold.
First 3 elements of x:
[  0 255   0]
First 1 elements of y:
[25]

In [ ]:
pxlib_path = base + "pxlib_rural_input_images/"
pxlib_directory = os.listdir("pxlib_rural_input_images/")

iou_with_pxlib = models_to_use(base, ground_truth_path, pxlib_path, ground_truth_directory, pxlib_directory)
# print(iou_with_pxlib)

In [ ]:
def format_models_and_iou(ground_truth_directory, iou_with_m_seg, iou_with_pxlib):
    '''Take in models with their mean iou values for each image
    
    ground_truth_directory -- list
    iou_with_m_seg -- list
    iou_with_pxlib -- list
    
    Return:
    table comparing iou values -- DataFrame
    '''
    
    ground_truth = "Ground Truth and "
    m_seg_column = ground_truth + "M Seg"
    pxlib_column = ground_truth + "PX Lib"
    
    data = {m_seg_column: iou_with_m_seg, pxlib_column: iou_with_pxlib}
    create_df = pd.DataFrame(data, index=[ground_truth_directory])
    
    
    return create_df

In [ ]:
format_models_and_iou(ground_truth_directory, iou_with_m_seg, iou_with_pxlib)